In [ ]:
import numpy as np
import sympy
import hls4ml

# Pass trained expressions to hls4ml converter and compile

In [ ]:
from hls4ml.utils.symbolic_utils import init_pysr_lut_functions
function_definitions = ['sin_lut(x) = math_lut(sin, x, N=256, range_start=-8, range_end=8)']
init_pysr_lut_functions(init_defaults=True, function_definitions=function_definitions)

lut_functions = {
    'sin_lut': {
        'math_func': 'sin',
        'range_start': -8,
        'range_end': 8,
        'table_size': 256
    }
}

expr_str = [
    'sin_lut(x0)',
           ]

expr = []
for i in range(len(expr_str)):
    expr.append(sympy.parsing.sympy_parser.parse_expr(expr_str[i]))

In [ ]:
hls_model = hls4ml.converters.convert_from_symbolic_expression(expr,
                                                               n_symbols=1,
                                                               output_dir='model-test/my-hls-test',
                                                               precision='ap_fixed<16,16>',
                                                               part='xcvu9p-flga2577-2-e',
                                                               lut_functions=lut_functions,
                                                               #use_built_in_lut_functions=True,
                                                               #hls_include_path='/data/containers/cmd/include',
                                                               #hls_libs_path='/data/containers/cmd/lnx64'
                                                               )

hls_model.write()
hls_model.compile()

In [ ]:
# Pass all precisions at once

In [ ]:
from pysr import PySRRegressor
from glob import glob
#pkl_dir="model_pol_80"
#pkl_dir="model_pol_40"
#pkl_dir="model_pol_20"
#pkl_dir="model_sin_80"
#pkl_dir="model_sin_40"
#pkl_dir="model_sin_20"
#pkl_dir="model_gau_80"
#pkl_dir="model_gau_40"
#pkl_dir="model_gau_20"
#pkl_dir="model_log_80"
#pkl_dir="model_log_40"
#pkl_dir="model_log_20"
#pkl_dir="model_cfg_off_40"
pkl_dir="model_cfg_on_16-6_40"
assert(len(glob(pkl_dir+'/*.pkl'))==1)
model_pysr = PySRRegressor.from_file(glob(pkl_dir+'/*.pkl')[0])

In [ ]:
print("Sympy simplified equations\n")
with sympy.evaluate(True):
    for i in range(5):
        print("Tagger {} = ".format(i)+str(model_pysr.sympy()[i])+"\n------------------------------------------")

In [ ]:
model_pysr.set_params(extra_sympy_mappings={"gauss": lambda x: sympy.exp(-x*x)})
model_pysr.refresh()
class gauss(sympy.Function):
    pass

In [ ]:
from hls4ml.utils.symbolic_utils import init_pysr_lut_functions
function_definitions = ['sin_lut(x) = math_lut(sin, x, N=256, range_start=-8, range_end=8)']
init_pysr_lut_functions(init_defaults=True, function_definitions=function_definitions)

lut_functions = {
    'sin_lut': {
        'math_func': 'sin',
        'range_start': -8,
        'range_end': 8,
        'table_size': 256
    }
}

expr_str_nonlut = []
expr_str_lut = []
expr_nonlut = []
expr_lut = []
for i in range(5):
    expr_str_nonlut.append(str(model_pysr.sympy()[i]))
    expr_str_lut.append(expr_str_nonlut[i].replace("sin", "sin_lut"))
    expr_nonlut.append(sympy.parsing.sympy_parser.parse_expr(expr_str_nonlut[i]))
    expr_lut.append(sympy.parsing.sympy_parser.parse_expr(expr_str_lut[i]))
    print('nonLUT = '+expr_str_nonlut[i])
    print('LUT    = '+expr_str_lut[i])
    print("----------------------------------------")

precision=['12,6','13,6','14,6','15,6','16,6','17,6','18,6']
hls_model_nonlut=[]
for i in range(len(precision)):
    hls_model_nonlut.append(hls4ml.converters.convert_from_symbolic_expression(expr_nonlut,
                                                               n_symbols=16,
                                                               output_dir=pkl_dir+"/hlstest-nonlut-{}".format(precision[i]),
                                                               precision='ap_fixed<{}>'.format(precision[i]),
                                                               part='xcvu9p-flga2577-2-e',
                                                               #lut_functions=lut_functions,
                                                               #use_built_in_lut_functions=True,
                                                               ))
    print(precision[i])
    hls_model_nonlut[i].write()
    hls_model_nonlut[i].compile()

hls_model_lut=[]
for i in range(len(precision)):
    hls_model_lut.append(hls4ml.converters.convert_from_symbolic_expression(expr_lut,
                                                               n_symbols=16,
                                                               output_dir=pkl_dir+"/hlstest-lut-{}".format(precision[i]),
                                                               precision='ap_fixed<{}>'.format(precision[i]),
                                                               part='xcvu9p-flga2577-2-e',
                                                               lut_functions=lut_functions,
                                                               #use_built_in_lut_functions=True,
                                                               ))
    print(precision[i])
    hls_model_lut[i].write()
    hls_model_lut[i].compile()


In [ ]:
hls_model.predict(np.array([3.000001], dtype=np.float32))
#hls_model_nonlut[0].predict(np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], dtype=np.float32))

In [ ]:
!cat my-hls-test/firmware/myproject.cpp 

# Compare PySR and hls4ml accuracy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_curve, auc, accuracy_score

In [ ]:
from sklearn.datasets import fetch_openml
data = fetch_openml('hls4ml_lhc_jets_hlf')
X, Y = data['data'].to_numpy(), data['target'].to_numpy()

print(data['feature_names'])
print(X.shape, Y.shape)
print(Y[:12])

LE = LabelEncoder()
Y = LE.fit_transform(Y)
print(Y[:12])

Y = to_categorical(Y, 5)
print(Y[:12])

Y=2*Y-1
print(Y[:12])

In [ ]:
train_ratio = 0.02
val_ratio = 0.5
test_ratio = 1 - train_ratio - val_ratio

# randomly partition data set into train/val/test sets with given ratios
X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size = test_ratio, random_state=15)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size = val_ratio/(val_ratio+train_ratio), random_state=15)

# standardize feature variables
scaler = StandardScaler().fit(X_train_val)
del X_train_val
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# print input data shapes
print("\nPartition the data set into train/val/test: %.2f/%.2f/%.2f" % (train_ratio, val_ratio, test_ratio))
print("X_train.shape: " + str(X_train.shape))
print("Y_train.shape: " + str(Y_train.shape))
print("X_val.shape: " + str(X_val.shape))
print("Y_val.shape: " + str(Y_val.shape))
print("X_test.shape: " + str(X_test.shape))
print("Y_test.shape: " + str(Y_test.shape))

In [ ]:
Y_pysr=model_pysr.predict(X_test)
auc_pysr=[]
print("pysr acc = {0:.3f}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pysr, axis=1))))
for x, label in enumerate(LE.classes_):
        fpr, tpr, _ = roc_curve(Y_test[:, x], Y_pysr[:, x])
        auc_pysr.append(auc(fpr, tpr))
print("auc = {0:.3f},{1:.3f},{2:.3f},{3:.3f},{4:.3f}".format(auc_pysr[0],auc_pysr[1],auc_pysr[2],auc_pysr[3],auc_pysr[4]))

In [ ]:
Y_hls_nonlut=[]
Y_hls_lut=[]
Y_pysr=model_pysr.predict(X_test)
print("pysr acc = {0:.3f}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pysr, axis=1))))
for i in range(len(precision)):
    Y_hls_nonlut.append(hls_model_nonlut[i].predict(X_test))
    Y_hls_lut.append(hls_model_lut[i].predict(X_test))
    auc_hls_nonlut=[]
    auc_hls_lut=[]
    for x, label in enumerate(LE.classes_):
        fpr_nonlut, tpr_nonlut, _ = roc_curve(Y_test[:, x], Y_hls_nonlut[i][:, x])
        fpr_lut, tpr_lut, _ = roc_curve(Y_test[:, x], Y_hls_lut[i][:, x])
        auc_hls_nonlut.append(auc(fpr_nonlut, tpr_nonlut))
        auc_hls_lut.append(auc(fpr_lut, tpr_lut))
    print("===========================")
    print("precision<{}>".format(precision[i]))
    print("nonLUT acc = {0:.3f}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_hls_nonlut[i], axis=1))))
    print("nonLUT auc = {0:.3f},{1:.3f},{2:.3f},{3:.3f},{4:.3f}".format(auc_hls_nonlut[0],auc_hls_nonlut[1],auc_hls_nonlut[2],auc_hls_nonlut[3],auc_hls_nonlut[4]))
    print("LUT acc    = {0:.3f}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_hls_lut[i], axis=1))))
    print("LUT auc    = {0:.3f},{1:.3f},{2:.3f},{3:.3f},{4:.3f}".format(auc_hls_lut[0],auc_hls_lut[1],auc_hls_lut[2],auc_hls_lut[3],auc_hls_lut[4]))

In [ ]:
import tensorflow as tf
from keras.layers import *
from qkeras import *
from qkeras.utils import load_qmodel
tf.get_logger().setLevel('ERROR')

model = load_qmodel('model_nn_qat/qkeras_model_ap_fixed_16-6')
config = hls4ml.utils.config_from_keras_model(model, granularity='model')
config['Model']['Precision'] = {'accum': 'ap_fixed<18,8>', 'default': 'ap_fixed<16,6>'}
hls_model_nn = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='model_nn_qat/hlstest_model_ap_fixed_16-6',
                                                       part='xcvu9p-flga2577-2-e')
hls_model_nn.write()
hls_model_nn.compile()
        
Y_hls_nn = hls_model_nn.predict(X_test)
        
print("acc = {0:.3f}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_hls_nn, axis=1))))

In [ ]:
Y_hls4ml=hls_model.predict(X_test)

In [ ]:
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_hls4ml, axis=1))))

In [ ]:
def plot_roc(y_test, y_pred, labels, model):
    color = ['blue','orange','green','red','purple']
    for x, label in enumerate(labels):        
        fpr, tpr, _ = roc_curve(y_test[:, x], y_pred[:, x])
        if model == 0:
            plt.plot(tpr, fpr, label='{0} QAT NN, AUC = {1:.1f}'.format(label, auc(fpr, tpr)*100.), linestyle=':', color=color[x], lw=1.3)
        if model == 1:
            plt.plot(tpr, fpr, label='{0}, sin, AUC = {1:.1f}'.format(label, auc(fpr, tpr)*100.), linestyle='-', color=color[x], lw=1.5)
        if model == 2:
            plt.plot(tpr, fpr, label='{0}, sin LUT, AUC = {1:.1f}'.format(label, auc(fpr, tpr)*100.), linestyle='--', color=color[x], lw=1.5)
    plt.semilogy()
    plt.xlabel("True positive rate", size=15, loc='right')
    plt.ylabel("False positive rate", size=15, loc='top')
    plt.tick_params(axis='both', which='major', direction='in', length=6, width=1.2, labelsize=12, right=True, top=True)
    plt.tick_params(axis='both', which='minor', direction='in', length=2, width=1, labelsize=12, right=True, top=True)
    plt.xlim(0, 1)
    plt.ylim(0.001, 1)
    plt.grid(True)
    plt.legend(loc='best', prop={'size': 10})  
    
plt.figure(figsize=(15, 15))
axes = plt.subplot(2,2,1)
#plot_roc(Y_test, Y_hls_nn, LE.classes_, 0)
plot_roc(Y_test, Y_hls_nonlut[0], LE.classes_, 1)
plot_roc(Y_test, Y_hls_lut[0], LE.classes_, 2)

#plt.savefig('HAA_bbtt_all.pdf',bbox_inches='tight')

In [ ]:
nbins=40
rmin=-4
rmax=4

fig, ax = plt.subplots(figsize = (14,14))
for k in range(16):
    ax = plt.subplot(4, 4, k+1)
    plt.hist(X_test[Y_test[:,0]==1,k], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g jet')
    plt.hist(X_test[Y_test[:,1]==1,k], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q jet')
    plt.hist(X_test[Y_test[:,2]==1,k], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t jet')
    plt.hist(X_test[Y_test[:,3]==1,k], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w jet')
    plt.hist(X_test[Y_test[:,4]==1,k], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z jet')
    #plt.legend(loc='best')
    #plt.title('MC truth g-jet')
    plt.xlabel('Input x{}'.format(k))
plt.legend(loc='best')

# Run synthesis from command line

In [ ]:
source /data/tools/Xilinx/Vivado/2020.1/settings64.sh
source /data/setup_xilinx_lic.sh

In [ ]:
!vivado_hls -f build_prj.tcl "reset=1 synth=1 csim=0 cosim=0 validation=0 export=0 vsynth=1"

In [ ]:
#hls4ml.report.read_vivado_report('my-hls-test/')
!cat my-hls-test/myproject_prj/solution1/syn/report/myproject_csynth.rpt

# Appendix: PySR training

In [ ]:
#export JULIA_NUM_THREADS=32
#from pysr import PySRRegressor

#class sin_lut(sympy.Function):
#    pass

#model_pysr = PySRRegressor.from_file("hall_exp.pkl")

In [ ]:
# sin
from pysr import PySRRegressor

model_pysr = PySRRegressor(
    model_selection="accuracy",
    niterations=80,
    timeout_in_seconds=60*60*1,
    maxsize=20,
    select_k_features=6,
    binary_operators=["+", "-", "*"],
    unary_operators=[
        "sin"
    ],
    nested_constraints={
        "sin": {"sin": 0}
    },
    loss="L2MarginLoss()" # (1 - y * y')^2
)

In [ ]:
# latency-aware training
from pysr import PySRRegressor
model_pysr = PySRRegressor(
    model_selection="accuracy",
    niterations=80,
    timeout_in_seconds=60*60*1,
    maxsize=20,
    select_k_features=6,
    binary_operators=["+", "-", "*"],
    unary_operators=[
        "log_abs(x)=log(abs(x))", "sin", "tan", "exp", "sinh", "cosh"
    ],
    constraints={
        "log_abs":  10,
        "sin":      10,
        "tan":     10,
        "exp":      10,
        "sinh":     10,
        "cosh":     10
    },
    complexity_of_operators={"*": 1, "+": 1, "-": 1, "log_abs": 4, "sin": 8, "tan": 48, "cosh": 8, "sinh": 9, "exp": 3},
    nested_constraints={
        "log_abs":  {"log_abs": 0, "sin": 0, "tan": 0, "exp": 0, "sinh": 0, "cosh": 0},
        "sin":      {"log_abs": 0, "sin": 0, "tan": 0, "exp": 0, "sinh": 0, "cosh": 0},
        "tan":      {"log_abs": 0, "sin": 0, "tan": 0, "exp": 0, "sinh": 0, "cosh": 0},
        "exp":      {"log_abs": 0, "sin": 0, "tan": 0, "exp": 0, "sinh": 0, "cosh": 0},
        "sinh":     {"log_abs": 0, "sin": 0, "tan": 0, "exp": 0, "sinh": 0, "cosh": 0},
        "cosh":     {"log_abs": 0, "sin": 0, "tan": 0, "exp": 0, "sinh": 0, "cosh": 0}
    },
    extra_sympy_mappings={
                          "log_abs": lambda x: sympy.log(abs(x))},
    loss="L2MarginLoss()" # (1 - y * y')^2
)

In [ ]:
model_pysr.fit(X_train, Y_train)

In [ ]:
print(model_pysr)

In [ ]:
print("Original PySR equations\n")
with sympy.evaluate(False):
    for i in range(5):
        print("Tagger {} = ".format(i)+str(model_pysr.sympy()[i])+"\n------------------------------------------")

In [ ]:
print("Sympy simplified equations\n")
for i in range(5):
    print("Tagger {} = ".format(i)+str(model_pysr.sympy()[i])+"\n------------------------------------------")

In [ ]:
print("Sympy expanded equations\n")
x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15 = sympy.symbols('x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15')
for i in range(5):
    print("Tagger {} = ".format(i)+str(sympy.expand(model_pysr.sympy()[i]))+"\n------------------------------------------")

In [ ]:
Y_pysr=model_pysr.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pysr, axis=1))))

In [ ]:
nbins=40
rmin=-2
rmax=2

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,0]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='g tagger', log=True)
plt.hist(Y_pysr[Y_test[:,0]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q tagger')
plt.hist(Y_pysr[Y_test[:,0]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t tagger')
plt.hist(Y_pysr[Y_test[:,0]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w tagger')
plt.hist(Y_pysr[Y_test[:,0]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z tagger')
plt.legend(loc='best')
plt.title('MC truth g-jet')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='g tagger', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q tagger')
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t tagger')
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w tagger')
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z tagger')
plt.legend(loc='best')
plt.title('MC truth g-jet')
plt.xlabel('hls4ml score')
plt.show()

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,1]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g tagger', log=True)
plt.hist(Y_pysr[Y_test[:,1]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='q tagger')
plt.hist(Y_pysr[Y_test[:,1]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t tagger')
plt.hist(Y_pysr[Y_test[:,1]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w tagger')
plt.hist(Y_pysr[Y_test[:,1]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z tagger')
plt.legend(loc='best')
plt.title('MC truth q-jet')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g tagger', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='q tagger')
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t tagger')
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w tagger')
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z tagger')
plt.legend(loc='best')
plt.title('MC truth q-jet')
plt.xlabel('hls4ml score')
plt.show()

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,2]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g tagger', log=True)
plt.hist(Y_pysr[Y_test[:,2]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q tagger')
plt.hist(Y_pysr[Y_test[:,2]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='t tagger')
plt.hist(Y_pysr[Y_test[:,2]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w tagger')
plt.hist(Y_pysr[Y_test[:,2]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z tagger')
plt.legend(loc='best')
plt.title('MC truth t-jet')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g tagger', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q tagger')
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='t tagger')
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w tagger')
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z tagger')
plt.legend(loc='best')
plt.title('MC truth t-jet')
plt.xlabel('hls4ml score')
plt.show()

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,3]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g tagger', log=True)
plt.hist(Y_pysr[Y_test[:,3]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q tagger')
plt.hist(Y_pysr[Y_test[:,3]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t tagger')
plt.hist(Y_pysr[Y_test[:,3]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='w tagger')
plt.hist(Y_pysr[Y_test[:,3]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z tagger')
plt.legend(loc='best')
plt.title('MC truth w-jet')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g tagger', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q tagger')
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t tagger')
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='w tagger')
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='z tagger')
plt.legend(loc='best')
plt.title('MC truth w-jet')
plt.xlabel('hls4ml score')
plt.show()

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,4]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g tagger', log=True)
plt.hist(Y_pysr[Y_test[:,4]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q tagger')
plt.hist(Y_pysr[Y_test[:,4]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t tagger')
plt.hist(Y_pysr[Y_test[:,4]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w tagger')
plt.hist(Y_pysr[Y_test[:,4]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='z tagger')
plt.legend(loc='best')
plt.title('MC truth z-jet')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='g tagger', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='q tagger')
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='t tagger')
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='w tagger')
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='z tagger')
plt.legend(loc='best')
plt.title('MC truth z-jet')
plt.xlabel('hls4ml score')
plt.show()

In [ ]:
nbins=40
rmin=-2
rmax=2

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,0]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth g-jet', log=True)
plt.hist(Y_pysr[Y_test[:,1]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth q-jet')
plt.hist(Y_pysr[Y_test[:,2]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth t-jet')
plt.hist(Y_pysr[Y_test[:,3]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth w-jet')
plt.hist(Y_pysr[Y_test[:,4]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth z-jet')
plt.legend(loc='best')
plt.title('g tagger')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth g-jet', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth q-jet')
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth t-jet')
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth w-jet')
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,0], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth z-jet')
plt.legend(loc='best')
plt.title('g tagger')
plt.xlabel('hls4ml score')
plt.show()

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,0]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth g-jet', log=True)
plt.hist(Y_pysr[Y_test[:,1]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth q-jet')
plt.hist(Y_pysr[Y_test[:,2]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth t-jet')
plt.hist(Y_pysr[Y_test[:,3]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth w-jet')
plt.hist(Y_pysr[Y_test[:,4]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth z-jet')
plt.legend(loc='best')
plt.title('q tagger')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth g-jet', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth q-jet')
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth t-jet')
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth w-jet')
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,1], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth z-jet')
plt.legend(loc='best')
plt.title('q tagger')
plt.xlabel('hls4ml score')
plt.show()

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,0]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth g-jet', log=True)
plt.hist(Y_pysr[Y_test[:,1]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth q-jet')
plt.hist(Y_pysr[Y_test[:,2]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth t-jet')
plt.hist(Y_pysr[Y_test[:,3]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth w-jet')
plt.hist(Y_pysr[Y_test[:,4]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth z-jet')
plt.legend(loc='best')
plt.title('t tagger')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth g-jet', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth q-jet')
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth t-jet')
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth w-jet')
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,2], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth z-jet')
plt.legend(loc='best')
plt.title('t tagger')
plt.xlabel('hls4ml score')
plt.show()

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,0]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth g-jet', log=True)
plt.hist(Y_pysr[Y_test[:,1]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth q-jet')
plt.hist(Y_pysr[Y_test[:,2]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth t-jet')
plt.hist(Y_pysr[Y_test[:,3]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth w-jet')
plt.hist(Y_pysr[Y_test[:,4]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth z-jet')
plt.legend(loc='best')
plt.title('w tagger')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth g-jet', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth q-jet')
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth t-jet')
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth w-jet')
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,3], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth z-jet')
plt.legend(loc='best')
plt.title('w tagger')
plt.xlabel('hls4ml score')
plt.show()

fig, ax = plt.subplots(figsize = (12,8))
ax = plt.subplot(2, 2, 1)
plt.hist(Y_pysr[Y_test[:,0]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth g-jet', log=True)
plt.hist(Y_pysr[Y_test[:,1]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth q-jet')
plt.hist(Y_pysr[Y_test[:,2]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth t-jet')
plt.hist(Y_pysr[Y_test[:,3]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth w-jet')
plt.hist(Y_pysr[Y_test[:,4]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth z-jet')
plt.legend(loc='best')
plt.title('z tagger')
plt.xlabel('PySR score')
ax = plt.subplot(2, 2, 2)
plt.hist(Y_hls_lut[0][Y_test[:,0]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth g-jet', log=True)
plt.hist(Y_hls_lut[0][Y_test[:,1]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth q-jet')
plt.hist(Y_hls_lut[0][Y_test[:,2]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth t-jet')
plt.hist(Y_hls_lut[0][Y_test[:,3]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='step', label='MC truth w-jet')
plt.hist(Y_hls_lut[0][Y_test[:,4]==1,4], bins=nbins, range=(rmin,rmax), density=1, histtype='stepfilled', alpha=0.2, label='MC truth z-jet')
plt.legend(loc='best')
plt.title('z tagger')
plt.xlabel('hls4ml score')
plt.show()